In [ ]:
# Hyperparameter grids
random_state_grid = [251, 582, 591, 825, 189, 888, 395, 943, 616, 896]
n_estimators_grid = [100]
max_depth_grid = [None]
n_features = df.shape[1] - 1

all_predictions_rf = []
all_predictions_features = []

for random_state in random_state_grid:
    rf_model = RandomForestRegressor(n_estimators=100,
                                      random_state=random_state, n_jobs=8, max_features=n_features//3)

    predictions_rf = []
    actual_values_rf = []

    tscv = TimeSeriesSplit(n_splits=30, gap=0)

    for train_index, test_index in tscv.split(df):
        train, test = df.iloc[train_index], df.iloc[test_index]
        X_train, y_train = train.drop(columns=['CPIAUCSL']), train['CPIAUCSL']
        X_test, y_test = test.drop(columns=['CPIAUCSL']), test['CPIAUCSL']

        rf_model.fit(X_train, y_train)
        y_pred_rf = rf_model.predict(X_test)

        predictions_rf.extend(y_pred_rf)
        actual_values_rf.extend(y_test)

    if not all_predictions_rf:
        all_predictions_rf = predictions_rf.copy()
    else:
        all_predictions_rf = [sum(x) for x in zip(all_predictions_rf, predictions_rf)]

# Average predictions over all random states
all_predictions_rf = np.array(all_predictions_rf) / len(random_state_grid)

#store to csv for np array
np.savetxt("all_predictions_rf.csv", all_predictions_rf, delimiter=",")

for random_state in random_state_grid:
    rf_model = RandomForestRegressor(n_estimators=100,
                                      random_state=random_state, n_jobs=8, max_features='sqrt')

    predictions_features = []
    actual_values_features = []

    tscv = TimeSeriesSplit(n_splits=30, gap=0)

    for train_index, test_index in tscv.split(df):
        train, test = df.iloc[train_index], df.iloc[test_index]
        X_train, y_train = train.drop(columns=['CPIAUCSL']), train['CPIAUCSL']
        X_test, y_test = test.drop(columns=['CPIAUCSL']), test['CPIAUCSL']

        rf_model.fit(X_train, y_train)
        y_pred_features = rf_model.predict(X_test)

        predictions_features.extend(y_pred_features)
        actual_values_features.extend(y_test)

    if not all_predictions_features:
        all_predictions_features = predictions_features.copy()
    else:
        all_predictions_features = [sum(x) for x in zip(all_predictions_features, predictions_features)]

# Average predictions over all random states
all_predictions_features = np.array(all_predictions_features) / len(random_state_grid)

# store to csv for np array
np.savetxt("all_predictions_features.csv", all_predictions_rf, delimiter=",")


# Ensure the length of plot_dates is the same as the length of predictions and actual values
plot_dates = plot_dates[:len(actual_values_rf)]




In [ ]:
# Plotting actual values and average predictions
plt.figure(figsize=(8, 6))
plt.plot(plot_dates, actual_values_rf, label='Actual', linestyle='--', color='blue')
plt.plot(plot_dates, all_predictions_rf, label='Predicted - RF/HSRF', color='orange')
plt.plot(plot_dates, all_predictions_features, label='Predicted - RF Features', color='green')
plt.xlabel('Year')
plt.ylabel('Inflation')
plt.legend()
plt.show()